In [1]:
#Importing require library
import pandas as pd
import selenium
import re
from selenium.webdriver.support.ui import WebDriverWait
from selenium import webdriver
import warnings
warnings.filterwarnings("ignore")
from selenium.common.exceptions import StaleElementReferenceException,NoSuchElementException,TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
import requests

# Q1 and Q2
Q1.Write a python program which searches all the product under a particular product from www.amazon.in.
The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search
for guitars.

Q2.In the above question, now scrape the following details of each product listed in first 3 pages of your
search results and save it in a data frame and csv. In case if any product has less than 3 pages in search
results then scrape all the products available under that product name. Details to be scraped are: "Brand
Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [2]:
#get product from user
product=input("Enter the name of product")

# connect to webdriver
driver=webdriver.Chrome(r"chromedriver.exe")

# search on google is https://www.amazon.in/
driver.get("https://www.amazon.in/") 

# enter the value for product
Product=driver.find_element(By.XPATH,"/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[2]/div[1]/input")
Product.send_keys(product)

# for click on search_button
search_button=driver.find_element(By.XPATH,'/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[3]/div/span/input')  
search_button.click()
time.sleep(3)

#scrape all links for products
links=[]
for x in range(0,3):
    link=driver.find_elements(By.XPATH,'//a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]')
    for i in link:
        links.append(i.get_attribute("href"))
        
    click_on_next_page=driver.find_element(By.XPATH,"//a[text()='Next']")
    click_on_next_page.click()
    time.sleep(3)
print(len(links))

#scrape require data from webpages
names=[]
brands=[]
prices=[]
returnss=[]
expexted_deliverys=[]
availabilitys=[]

for i in links:
    try:
        driver.get(i)
        try:
            name=driver.find_element(By.XPATH,'//h1[@id="title"]/span')
            names.append(name.text)
        except NoSuchElementException:
            names.append('-')

        try:
            brand=driver.find_element(By.XPATH,'//div[@class="a-section a-spacing-none"]/a')
            brands.append(brand.text)
        except NoSuchElementException:
            brands.append('-')

        try:
            price=driver.find_element(By.XPATH,'//div[@class="a-section a-spacing-none aok-align-center"]/span[2]')
            prices.append(price.text.replace('\n','.'))
        except NoSuchElementException:
            prices.append('-')

        try:
            returns=driver.find_element(By.XPATH,'//span[@class="a-declarative"]/div/a')
            returnss.append(returns.text)
        except NoSuchElementException:
            returnss.append('-')

        try:
            expexted_delivery=driver.find_element(By.XPATH,'//div[@class="a-spacing-base"]/span/span[1]')
            expexted_deliverys.append(expexted_delivery.text)
        except NoSuchElementException:
            expexted_deliverys.append('-')

        try:
            availability=driver.find_element(By.XPATH,'//span[@class="a-size-medium a-color-success"]')
            availabilitys.append(availability.text)
        except NoSuchElementException:
            availabilitys.append('-')
    except TimeoutError:
        names.append('-')
        brands.append('-')
        prices.append('-')
        returnss.append('-')
        expexted_deliverys.append('-')
        availabilitys.append('-')
    time.sleep(1)
    
#creating dataframe
df=pd.DataFrame({
    'name':names,
    'brand':brands,
    'price':prices,
    'returns':returnss,
    'expexted_delivery':expexted_deliverys,
    'availability':availabilitys
})
df

Enter the name of productguitar
190


,name,brand,price,returns,expexted_delivery,availability
0,Kadence Frontier guitar with Online Guitar lea...,Visit the Kadence Store,"₹4,949.00",7 days Replacement,"Monday, 19 December",
1,Kadence Slowhand Premium Jumbo Semi Acoustic G...,Visit the Kadence Store,"₹9,900.00",7 days Replacement,"Wednesday, 21 December",
2,Vault DA20 Dreadnought Acoustic Guitar With Gi...,Visit the VAULT Store,"₹4,465.00",7 days Replacement,"Tuesday, 20 December",In stock.
3,VD ENTERPRISE String Acoustic Guitar | Adjusta...,Brand: VD ENTERPRISE,₹799.00,7 days Replacement,"Monday, 19 December",In stock.
4,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C ...",Visit the JUAREZ Store,"₹2,278.10",7 days Replacement,"Tuesday, 20 December",In stock.
...,...,...,...,...,...,...
185,"Kadence Guitar Frontier Series, Blue Acoustic ...",Visit the Kadence Store,"₹4,999.00",7 days Replacement,"Wednesday, 21 December",In stock.
186,"Yamaha F370 Acoustic Guitar, Natural",Visit the YAMAHA Store,"₹12,990.00",7 days Replacement,December 19 - 21,Only 1 left in stock.
187,"Yamaha F370 Acoustic Guitar, Natural",Visit the YAMAHA Store,"₹12,990.00",7 days Replacement,December 19 - 21,Only 1 left in stock.
188,"Yamaha F370 Acoustic Guitar, Natural",Visit the YAMAHA Store,"₹12,990.00",7 days Replacement,December 19 - 21,Only 1 left in stock.


# Q3
Write a python program to access the search bar and search button on images.google.com and scrape 10
images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’.

In [41]:
#connect to the webdriver
driver=webdriver.Chrome(r"chromedriver.exe")
driver.maximize_window()

# find on google ,https://images.google.com/
driver.get("https://images.google.com/")
time.sleep(2)

#search for keyword for open site we can use any
search=driver.find_element(By.XPATH,'//input[@class="gLFyf"]')
search.send_keys('Images')
time.sleep(1)

#click on search button
Click=driver.find_element(By.XPATH,'//span[@class="z1asCe MZy1Rb"]')
Click.click()
time.sleep(2)

m=1 #this is use for change name when image will download otherwise it is overlaped
keyword=['fruits','cars','Machine Learning','Guitar','Cakes']
for key in keyword:
    search=driver.find_element(By.XPATH,'//input[@class="og3lId"]')
    search.clear()
    search.send_keys(str(key))
    time.sleep(3)
    
    Click=driver.find_element(By.XPATH,'//span[@class="n6h3Rc"]')
    Click.click()
    time.sleep(3)
    
    x=0
    while x<10:
        driver.execute_script("window.scrollBy(0,500)")
        time.sleep(1)
        x=x+1
    
    images=driver.find_elements(By.XPATH,'//img[@class="rg_i Q4LuWd"]')
    urls=[]
    for i in images:
        X=i.get_attribute('src')
        if X is not None:
            if(X[0:4]=='http'):
                urls.append(X)
    i=0
    for i in range(len(urls)):
        if i>10:
            break
        response=requests.get(urls[i])
        file=open(r"D:"+str(i+m)+".jpg","wb")
        file.write(response.content)
        m=m+1

Output of Question 3 is not shown here because downloaded Images is in my D drive 

# Q4
Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on
www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be
scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,
“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the
details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [6]:
#get product from user
product=input("Enter the name of product")

# connect to webdriver
driver=webdriver.Chrome(r"chromedriver.exe")

# search on google is https://www.flipkart.com/
driver.get("https://www.flipkart.com/") 

# for click on login_page for close it
login_page=driver.find_element(By.XPATH,'//button[@class="_2KpZ6l _2doB4z"]')  
login_page.click()
time.sleep(1)

# enter the value for product
Product=driver.find_element(By.CLASS_NAME,'_3704LK')
Product.send_keys(product)

# for click on search_button
search_button=driver.find_element(By.XPATH,'/html/body/div[1]/div/div[1]/div[1]/div[2]/div[2]/form/div/button')
search_button.click()
time.sleep(3)

#fetch link for product in first page
url=driver.find_elements(By.XPATH,'//a[@class="_1fQZEK"]')
urls=[]
for i in url:
    urls.append(i.get_attribute("href"))
print(len(urls))



#Fetch all require Data
brand_namess=[]
smartphone_names=[]
colours=[]
rams=[]
storages=[]
primarys=[]
secondarys=[]
displays=[]
batterys=[]
prices=[]
for i in urls:
    try:
        driver.get(i)
        clickonReadmore=driver.find_element(By.XPATH,'//button[@class="_2KpZ6l _1FH0tX"]')
        clickonReadmore.click()
        time.sleep(2)
        
        try:
            brand_names=driver.find_element(By.XPATH,'//span[@class="B_NuCI"]')
            brand_namess.append(brand_names.text)
        except NoSuchElementException:
            brand_namess.append('-')
        
        try:
            smartphone_name=driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][1]/table/tbody/tr[3]/td[2]/ul/li')
            smartphone_names.append(smartphone_name.text)
        except NoSuchElementException:
            smartphone_names.append('-')
            
        try:
            colour=driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][1]/table/tbody/tr[4]/td[2]/ul/li')
            colours.append(colour.text)
        except NoSuchElementException:
            colours.append('-')
        
        try:
            ram=driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][4]/table/tbody/tr[2]/td[2]/ul/li')
            rams.append(ram.text)
        except NoSuchElementException:
            rams.append('-')
        
        try:
            storage=driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][4]/table/tbody/tr[1]/td[2]/ul/li')
            storages.append(storage.text)
        except NoSuchElementException:
            storages.append('-')
            
        try:
            primary=driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table/tbody/tr[2]/td[2]/ul/li')
            primarys.append(primary.text)
        except NoSuchElementException:
            primarys.append('-')
        
        try:
            secondary=driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][5]/table/tbody/tr[6]/td[2]/ul/li')
            secondarys.append(secondary.text)
        except NoSuchElementException:
            secondarys.append('-')
            
        try:
            display=driver.find_element(By.XPATH,'//div[@class="_3k-BhJ"][2]/table/tbody/tr[1]/td[2]/ul/li')
            displays.append(display.text)
        except NoSuchElementException:
            displays.append('-')
            
        try:
            battery=driver.find_element(By.XPATH,'//div[@class="_2418kt"]/ul/li[4]')
            batterys.append(battery.text)
        except NoSuchElementException:
            batterys.append('-')
            
        try:
            price=driver.find_element(By.XPATH,'//div[@class="_30jeq3 _16Jk6d"]')
            prices.append(price.text)
        except NoSuchElementException:
            prices.append('-')
            
            
    except TimeoutException:
        brand_namess.append('-')
        smartphone_names.append('-')
        colours.append('-')
        rams.append('-')
        storages.append('-')
        primarys.append('-')
        secondarys.append('-')
        displays.append('-')
        batterys.append('-')
        prices.append('-')
    time.sleep(1)
    
#Creating DataFrame
df=pd.DataFrame({
    'brand_names':brand_namess,
    'smartphone_name':smartphone_names,
    'colour':colours,
    'ram':rams,
    'storage':storages,
    'primary':primarys,
    'secondary':secondarys,
    'display':displays,
    'battery':batterys,
    'price':prices
})
df


Enter the name of productOneplus Nord
24


,brand_names,smartphone_name,colour,ram,storage,primary,secondary,display,battery,price
0,"OnePlus Nord CE 2 Lite 5G (Black Dusk, 128 GB)...",Nord CE 2 Lite 5G,Black Dusk,6 GB,128 GB,Primary & Secondary Camera,-,16.74 cm (6.59 inch),5000 mAh Battery,"₹19,998"
1,"OnePlus Nord CE 2 Lite 5G (Blue Tide, 128 GB) ...",Nord CE 2 Lite 5G,Blue Tide,6 GB,128 GB,Primary Camera,-,16.74 cm (6.59 inch),5000 mAh Battery,"₹19,990"
2,"OnePlus Nord CE 2 Lite 5G (Blue Tide, 128 GB) ...",Nord CE 2 Lite 5G,Blue Tide,6 GB,128 GB,64MP Rear Camera,Yes,16.74 cm (6.59 inch),5000 mAh Battery,"₹19,989"
3,"OnePlus Nord 2T 5G (Gray Shadow, 256 GB) (12 ...",Nord 2T 5G,Gray Shadow,12 GB,256 GB,Primary & Secondary Camera,-,16.94 cm (6.67 inch),4500 mAh Battery,"₹33,998"
4,"OnePlus Nord CE 2 Lite 5G (Blue Tide, 128 GB) ...",Nord CE 2 Lite 5G,Blue Tide,6 GB,128 GB,Primary & Secondary Camera,-,16.74 cm (6.59 inch),5000 mAh Battery,"₹19,899"
5,"OnePlus Nord CE 2 5G (Bahama Blue, 128 GB) (8...",Nord CE 2 5G,Bahama Blue,8 GB,128 GB,Primary Camera,-,16.33 cm (6.43 inch),4500 mAh Battery,"₹24,998"
6,"OnePlus Nord CE 2 Lite 5G (Black Dusk, 128 GB)...",Nord CE 2 Lite 5G,Black Dusk,6 GB,128 GB,Primary & Secondary Camera,-,16.74 cm (6.59 inch),5000 mAh Battery,"₹19,999"
7,"OnePlus Nord 2T 5G (Jade Fog, 256 GB) (12 GB ...",Nord 2T 5G,Jade Fog,12 GB,256 GB,Primary & Secondary Camera,-,17.02 cm (6.7 inch),4500 mAh Battery,"₹33,999"
8,"OnePlus Nord 2T 5G (Gray Shadow, 128 GB) (8 G...",Nord 2T 5G,Gray Shadow,8 GB,128 GB,Primary Camera,-,17.02 cm (6.7 inch),4500 mAh Battery,"₹28,990"
9,"OnePlus Nord 2T 5G (Jade Fog, 128 GB) (8 GB RAM)",Nord 2T 5G,Jade Fog,8 GB,128 GB,Primary & Secondary Camera,-,17.02 cm (6.7 inch),4500 mAh Battery,"₹28,899"


In [7]:
df.to_csv('Question 4')

# Q5
Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google
maps.

In [11]:
#Enter City name for find latitude, longitude
City = input("Enter the city name: ")

#connect to the webdriver
driver=webdriver.Chrome(r"chromedriver.exe")

# find on google ,google maps 
driver.get("https://www.google.co.in/maps")
time.sleep(2)

#Write city name which is enter by user
search_title = driver.find_element(By.ID,'searchboxinput')
search_title.send_keys(City)

#Click on search button for find city on google map
search_btn = driver.find_element(By.XPATH,'//div[@class="pzfvzf"]')
search_btn.click()
time.sleep(2)

try:
    #get url current url or link
    link = driver.current_url
    print("Current Link  is:",link)
    #now apply regex for filter url or link after @ sign
    filter_link = re.findall(r'@(.*)',link)
    #now split it in a parts so we can Identify it
    x=filter_link[0].split(',')
    #now zeroth index of x is a latitude and first index is longitude
    print('The latitude of this',City,'is',x[0])
    print('The longitude of this',City,'is',x[1])
    
except Exception as e:
    print("Error:",str(e))

Enter the city name:Rajkot
Current Link  is: https://www.google.co.in/maps/place/Rajkot,+Gujarat/@22.273927,70.8048529,14z/data=!4m5!3m4!1s0x3959c98ac71cdf0f:0x76dd15cfbe93ad3b!8m2!3d22.3038945!4d70.8021599
The latitude of this Rajkot is 22.273927
The longitude of this Rajkot is 70.8048529


# Q6
Write a program to scrap details of all the funding deals for second quarter (i.e Jan 21 – March 21)
from trak.in

This question is skipped because in doubt clearing session sir has told us to skip this question beacuse the website is not shows funding details

# Q7
Write a program to scrap all the available details of best gaming laptops from digit.in.

In [12]:
#connect to the webdriver
driver=webdriver.Chrome(r"chromedriver.exe")
driver.maximize_window()
# find on google ,https://www.digit.in/ 
driver.get("https://www.digit.in/")
time.sleep(2)

#Search for Laptop
search_Laptop = driver.find_element(By.XPATH,'//div[@class="menu"]/ul/li[3]/a')
search_Laptop.click()
time.sleep(0.5)

# #Search for best gaming laptops
search_gaming = driver.find_element(By.XPATH,"//a[text()='Best Gaming Laptops in India']")
search_gaming.click()
time.sleep(2)


#Fetch all require data from webpage
LaptopName =[]
OS=[]
display=[]
processor=[]
memory=[]
weight_dimension=[]
graphics=[]
description=[]

try:
    
    laptop=driver.find_elements(By.XPATH,'//div[@class="left_side"]/a')
    for i in laptop:
        LaptopName.append(i.text)
except NoSuchElementException:
    pass

try:
    
    os=driver.find_elements(By.XPATH,'//div[@class="Spcs-details"]/table/tbody/tr[5]/td[3]')
    for i in os:
        OS.append(i.text)
except NoSuchElementException:
    pass

try:
    
    displays=driver.find_elements(By.XPATH,'//div[@class="Spcs-details"]/table/tbody/tr[4]/td[3]')
    for i in displays:
        display.append(i.text)
except NoSuchElementException:
    pass

try:
    
    processors=driver.find_elements(By.XPATH,'//div[@class="Spcs-details"]/table/tbody/tr[3]/td[3]')
    for i in processors:
        processor.append(i.text)
except NoSuchElementException:
    pass

try:
    
    memorys=driver.find_elements(By.XPATH,'//div[@class="Spcs-details"]/table/tbody/tr[6]/td[3]')
    for i in memorys:
        memory.append(i.text)
except NoSuchElementException:
    pass

try:
    
    graphicss=driver.find_elements(By.XPATH,'//div[@class="Spcs-details"]/table/tbody/tr[7]/td[3]')
    for i in graphicss:
        graphics.append(i.text)
except NoSuchElementException:
    pass

try:
    
    weights_dimensions=driver.find_elements(By.XPATH,'//div[@class="Spcs-details"]/table/tbody/tr[8]/td[3]')
    for i in weights_dimensions:
        weight_dimension.append(i.text)
except NoSuchElementException:
    pass

try:
    
    descriptions=driver.find_elements(By.XPATH,'//div[@class="Section-center"]/div[2]')
    for i in descriptions:
        description.append(i.text)
except NoSuchElementException:
    pass


#Creating DataFrame
df=pd.DataFrame({
    'LaptopName':LaptopName,
    'OS':OS,
    'display':display,
    'processor':processor,
    'memory':memory,
    'weight_dimension':weight_dimension,
    'graphics':graphics,
    'description':description
})

df

,LaptopName,OS,display,processor,memory,weight_dimension,graphics,description
0,MSI Titan GT77-12UHS,Windows 11 Home,"17.3″ (3840 x 2160) screen, 120 Hz refresh rate",12th Gen Intel Core i9-12900HX 16 core processor,64 GB DDR5 RAM & 2 TB SSD,397 x 330 x 23 mm dimension & 3.3 kg weight,16 GB DDR6 NVIDIA GeForce RTX 3080 Ti Graphics...,The very best MSI gaming laptop featuring an I...
1,Alienware X17 R2,Windows 11 Home,"17.3″ (1920 x 1080) screen, 360 Hz refresh rate",12th Gen Intel Core i9-12900H 14 core processo...,32 GB DDR5 RAM & 1 TB SSD,299 x 359 x 21 mm dimension & 2.96 kg weight,16 GB DDR6 NVIDIA GEFORCE RTX 3080 Ti Graphics...,"Alienware, the premium gaming brand from Dell ..."
2,Acer Predator Triton 500 SE PT516-52s,Windows 11,16″ (2560 x 1600) screen,12th Gen Intel Core i7-12700H 14 core processo...,32 GB DDR5 RAM & 2 TB SSD,262 x 359 x 20 mm dimension & 2.4 kg weight,8 GB DDR6 NVIDIA GeForce RTX 3070 Ti Graphics ...,The Predator lineup from Acer has shown great ...
3,Omen By HP (16-B1371TX),Windows 11 Home,16.1″ (2560 x 1440) screen,12th Gen Intel Core i7-12700H 14 core processo...,8 GB DDR5 RAM & 1 TB SSD,369 x 248 x 23 mm dimension & 2.32 kg weight,8 GB GDDR6 NVIDIA GeForce RTX 3070 Graphics card,Omen by HP is a premium gaming lineup which is...
4,Acer Predator Helios 300 AN515-45,Windows 11 Home,"15.6″ (2560 x 1440) screen, 165 Hz refresh rate",AMD Ryzen 9-5900HX 8 core processor,16 GB DDR4 RAM & 512 GB SSD,363 x 255 x 23.9 mm dimension & 2.4 kg weight,8 GB DDR6 NVIDIA GeForce RTX 3070 Graphics card,Acer Predator Helios 300 AN515-45 is the best ...
5,MSI Delta 15 (A5EFK-083IN),Windows 11 Home,"15.6″ (1920 x 1080) screen, 240Hz refresh rate",AMD 5th Gen Ryzen 9-5900HX 8 core processor wi...,16 GB DDR4 RAM & 1 TB SSD,357 x 247 x 19 mm dimension & 1.9 kg weight,10 GB DDR6 AMD Radeon RX 6700M Graphics card,The best MSI gaming laptop with an AMD process...
6,Omen By HP (16-C0141AX),Windows 11 Home,16.1″ (2560 x 1440) screen,AMD Ryzen™ 9 5900HX 8 core processor with 3.3 ...,16 GB DDR4 RAM & 1 TB NVMe,36.92 x 24.8 x 2.3 mm dimension & 2.3 kg weight,8 GB GDDR6 AMD Radeon™ RX 6600M Graphics card,Another worthy gaming laptop from the Omen lin...
7,Lenovo Legion 5i Pro (82RF00MGIN),Windows 11 Home,"16″ (2560 x 1600) screen, 165 Hz refresh rate",12th Gen Intel Core i7-12700H 14 core processo...,16 GB DDR5 RAM & 1 TB SSD,360 x 264 x 20 mm dimension & 2.49 kg weight,6 GB DDR6 NVIDIA GeForce RTX 3060 Graphics card,Lenovo has gone a little mainstream with their...
8,Alienware M15 R5 Ryzen Edition Icc-C780001win,Windows 11 Home,"15.6″ (1920 x 1080) screen, 165Hz refresh rate",AMD Ryzen R7-5800H 8 core processor with 4.40 ...,16 GB DDR4 RAM & 512 GB SSD,356.2 x 272.5 x 22.85 mm dimension & 2.69 kg w...,6 GB DDR6 NVIDIA GeForce RTX 3060 Graphics card,Dell not only has gaming laptops under the Del...
9,Lenovo Slim 7 Gen 6 (82K8002JIN),Windows 11 Home,"15.6″ screen, 165 Hz refresh rate",Lenovo Ryzen 7-5800H processor with 3.2 GHz cl...,16 GB DDR4 RAM & 1 TB SSD,356 x 252 x 16 mm dimension & 1.9 kg weight,6 GB DDR6 NVIDIA GeForce 3060 Max-Q Graphics card,Lenovo has been pushing for thinner gaming lap...


# Q8
Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be
scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [32]:
#connect to the webdriver
driver=webdriver.Chrome(r"chromedriver.exe")
driver.maximize_window()

# find on google ,https://www.forbes.com/ 
driver.get("https://www.forbes.com/")


#Open lists
lists = driver.find_element(By.XPATH,'/html/body/div[1]/header/nav/div[1]/div/div/div[1]')
lists.click()


#Open lists for billionaires 
billionaires = driver.find_element(By.XPATH,'/html/body/div[1]/header/nav/div[1]/div/div/div[2]/ul/li[1]/div[1]/span')
billionaires.click()


#Open worls billionaires 
world_billionaires = driver.find_element(By.XPATH,'/html/body/div[1]/header/nav/div[1]/div/div/div[2]/ul/li[1]/div[2]/div[3]/ul/li[1]/a')
world_billionaires.click()

In [39]:
#fetch require data from datapage
rank=[]
name=[]
net_worth=[]
age=[]
citizenship=[]
source=[]
industry=[]

for x in range(0,200):
    try:
        ranks = driver.find_elements(By.XPATH,'//div[@class="rank"]')
        for i in ranks:
            rank.append(i.text)
        
        names=driver.find_elements(By.XPATH,'//div[@class="personName"]/div')
        for i in names:
            name.append(i.text)
            
        net_worths=driver.find_elements(By.XPATH,'//div[@class="netWorth"]/div[1]')
        for i in net_worths:
            net_worth.append(i.text)
        
        ages=driver.find_elements(By.XPATH,'//div[@class="age"]/div')
        for i in ages:
            age.append(i.text)
        
        citizenships=driver.find_elements(By.XPATH,'//div[@class="countryOfCitizenship"]')
        for i in citizenships:
            citizenship.append(i.text)
        
        sources=driver.find_elements(By.XPATH,'//div[@class="source"]')
        for i in sources:
            source.append(i.text)
        
        industrys=driver.find_elements(By.XPATH,'//div[@class="category"]//div')
        for i in industrys:
            industry.append(i.text)
            
        
        clicknxt=driver.find_element(By.XPATH,'//button[@class="pagination-btn pagination-btn--next "]')
        clicknxt.click()
        time.sleep(1)
    except NoSuchElementException:
        break
    
#Creating DataFrame
df=pd.DataFrame({
    'Rank':rank,
    'Name':name,
    'Net_worth':net_worth,
    'Age':age,
    'Citizenship':citizenship,
    'Source':source,
    'Industry':industry
})
df

,Rank,Name,Net_worth,Age,Citizenship,Source,Industry
0,1.,Elon Musk,$219 B,50,United States,"Tesla, SpaceX",Automotive
1,2.,Jeff Bezos,$171 B,58,United States,Amazon,Technology
2,3.,Bernard Arnault & family,$158 B,73,France,LVMH,Fashion & Retail
3,4.,Bill Gates,$129 B,66,United States,Microsoft,Technology
4,5.,Warren Buffett,$118 B,91,United States,Berkshire Hathaway,Finance & Investments
...,...,...,...,...,...,...,...
2663,2578.,Zhang Yuqiang,$1 B,66,China,Fiberglass,Manufacturing
2664,2578.,Zhou Ruxin,$1 B,59,China,Navigation,Technology
2665,2578.,Wen Zhou & family,$1 B,57,China,chemicals,Manufacturing
2666,2578.,Zhou Yifeng & family,$1 B,43,China,liquefied petroleum gas,Energy


# Q9
Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted
from any YouTube Video.

In [24]:
#connect to the webdriver
driver=webdriver.Chrome(r"chromedriver.exe")
driver.maximize_window()

# find on google ,https://www.youtube.com/watch?v=BBAyRBTfsOU
driver.get("https://www.youtube.com/watch?v=BBAyRBTfsOU")


#for load all the comments 
i = 0
while(i<150):
    driver.execute_script("window.scrollBy(0,500)")
    i+=1
    time.sleep(1.2)
print('Scroll is completed')

#Fetch like,cmnt and time
try:
    time=driver.find_elements(By.XPATH,'//yt-formatted-string[@class="published-time-text style-scope ytd-comment-renderer"]/a')
    times=[]
    for i in time[0:500]:
        times.append(i.text)
except NoSuchElementException:
    times.append('-')

try:
    cmnt=driver.find_elements(By.XPATH,'//yt-formatted-string[@id="content-text"]')
    cmnts=[]
    for i in cmnt[0:500]:
        cmnts.append(i.text)
except NoSuchElementException:
    cmnts.append('-')
    
try:
    up=driver.find_elements(By.XPATH,'//span[@id="vote-count-middle"]')
    ups=[]
    for i in up[0:500]:
        ups.append(i.text)
except NoSuchElementException:
    ups.append('-')
    
#Create DataFrame
df=pd.DataFrame({
    'Comments':cmnts,
    'Comment_Upvote':ups,
    'Time':times
})
df

Scroll is completed


,Comments,Comment_Upvote,Time
0,Vaaste is soo close to my heart and now we’re ...,180K,2 years ago
1,Let me know how many notice Sumbul in this song,514,2 weeks ago
2,Sumbul has Magical Eternal Presence \nEverywhe...,39,2 days ago (edited)
3,3 years passed but still masterpiece ️,6,1 day ago
4,1+ billion views...this song deserves more and...,8,1 day ago (edited)
...,...,...,...
495,Love song️️️,,1 month ago
496,Wow,,6 days ago
497,My fevreet song,,1 month ago
498,Sumbul...,1,2 days ago


# Q10
Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in
“London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews,
overall reviews, privates from price, dorms from price, facilities and property description.

In [3]:
#connect to the webdriver
driver=webdriver.Chrome(r"chromedriver.exe")
driver.maximize_window()

# find on google ,https://www.hostelworld.com/
driver.get("https://www.hostelworld.com/")
time.sleep(2)

#Click on bar
bar=driver.find_element(By.XPATH,'//input[@id="location-text-input-field"]')
bar.click()
time.sleep(2)

#search london
search=driver.find_element(By.XPATH,'//input[@id="search-input-field"]')
search.send_keys("London")
time.sleep(5)

#Click on london name
london=driver.find_element(By.XPATH,'//ul[@id="predicted-search-results"]/li[2]/div')
london.click()
time.sleep(2)

#Click on let's go button
clickk=driver.find_element(By.XPATH,'//div[@class="search-button"]')
clickk.click()
time.sleep(5)


# scrape data for distance,links and price
distances=[]
links=[]
price=[]
for X in range(0,4):
    # scroll page
    x=0
    while x<16:
        driver.execute_script("window.scrollBy(0,500)")
        time.sleep(1)
        x=x+1
    print("scrolled done",X)
    
    try:
        distance=driver.find_elements(By.XPATH,'//div[@class="subtitle body-3"]/a/span[1]')
        for i in distance:
            distances.append(i.text)
    except NoSuchElementException:
        distances.append('-')

    try:
        link=driver.find_elements(By.XPATH,'//div[@class="property"]/div/h2/a')
        for i in link:
            links.append(i.get_attribute('href'))
    except NoSuchElementException:
        links.append('-')
    try:
        for p_form in driver.find_elements(By.XPATH,"(//div[@class='price-col'])"):
            price.append(p_form.text)
    except NoSuchElementException:
        price.append('No')
        
    try:
        m=driver.find_element(By.XPATH,'//div[@class="pagination-item pagination-next"]')
        m.click()
    except NoSuchElementException:
        break

        
#the price of private and dorms in single list this code is by forget it 
privatesform=[]
dormsform=[]
for private in range(0,len(price),2):
    privatesform.append(price[private])
for dorms in range(1,len(price),2):
    dormsform.append(price[dorms])
    
# remove unwanted details from price lists     
pvr=[]
dvr=[]
for i in privatesform:
    if i=='No Privates Available':
        pvr.append(i)
    else:
        pvr.append(i[-7:].strip(' ').strip('\n'))
    
for i in dormsform:
    if i=='No Dorms Available':
        dvr.append(i)
    else:
        dvr.append(i[-7:].strip(' ').strip('\n').strip('m\n'))
        
        
print('Part one completed')


#scrape data for  property_descriptions,names,ratings,total_reviews,overall_reviews
property_descriptions=[]
names=[]
ratings=[]
total_reviews=[]
overall_reviews=[]
for x in links:
    try:
        driver.get(x)
        time.sleep(3)
        
        x=0
        while x<10:
            driver.execute_script("window.scrollBy(0,250)")
            time.sleep(1)
            x=x+1
    
        try:
            property_description=driver.find_element(By.XPATH,'//div[@class="description-container"]/div/div[2]')
            property_descriptions.append(property_description.text)
        except NoSuchElementException:
            property_descriptions.append('-')

        try:
            name=driver.find_element(By.XPATH,'//div[@class="title-2"]')
            names.append(name.text)
        except NoSuchElementException:
            names.append('-')

        try:
            rating=driver.find_element(By.XPATH,'//div[@class="score orange big"]')
            ratings.append(rating.text)
        except NoSuchElementException:
            ratings.append('-')

        try:
            total_review=driver.find_element(By.XPATH,'//div[@class="reviews"]')
            total_reviews.append(total_review.text.strip('Total Reviews'))
        except NoSuchElementException:
            total_reviews.append('-')

        try:
            overall_review=driver.find_element(By.XPATH,'//div[@class="keyword"]')
            overall_reviews.append(overall_review.text)
        except NoSuchElementException:
            overall_reviews.append('-')
        
    except TimeoutException:
        property_descriptions.append('-')
        names.append('-')
        ratings.append('-')
        total_reviews.append('-')
        overall_reviews.append('-')
        prs.append('-')
        drs.append('-')
        
# Creating DataFrame
df=pd.DataFrame({
    'names':names,
    'distances':distances,
    'ratings':ratings,
    'total_reviews':total_reviews,
    'privates':pvr,
    'dorms':dvr,
    'overall_reviews':overall_reviews,
    'property_descriptions':property_descriptions
    
})
df

scrolled done 0
scrolled done 1
scrolled done 2
Part one completed


,names,distances,ratings,total_reviews,privates,dorms,overall_reviews,property_descriptions
0,Wombat's City Hostel London,Hostel - 3.6km from city centre,8.9,14409,Rs9806,Rs3121,Fabulous,A safe haven in the middle of the metropolis: ...
1,St Christopher's Village,Hostel - 1.8km from city centre,8.0,11760,Rs9229,Rs2651,Fabulous,Hostelworld visitors have said St Christopher'...
2,Astor Hyde Park,Hostel - 4.3km from city centre,8.7,11810,No Privates Available,Rs2225,Fabulous,Note: 18-40 yrs old only. Astor Hyde Park won ...
3,Hostel One Notting Hill,Hostel - 5.5km from city centre,9.3,1631,Rs10908,Rs2855,Superb,The perfect place for solo travelers to connec...
4,Generator London,Hostel - 3km from city centre,7.7,7290,Rs10833,Rs2185,Very Good,Hostelworld guests have said Generator Hostel ...
...,...,...,...,...,...,...,...,...
56,Hanover Hotel,Hotel - 2.1km from city centre,-,0,Rs16756,No Dorms Available,No Rating,All the main attractions of the city are withi...
57,Beaconsfield,Hotel - 8.6km from city centre,-,0,Rs9002,No Dorms Available,No Rating,"Beaconsfield is a basic, budget hotel, situate..."
58,Hyde Park Court Hotel,Hotel - 4.3km from city centre,-,139,Rs11632,No Dorms Available,Rating,The Hyde Park Court Hotel is a centrally locat...
59,Park Hotel,Hotel - 4.9km from city centre,-,0,Rs15509,No Dorms Available,No Rating,"A friendly, family run hotel offering comforta..."
